In [ ]:
def draw_confusion_matrix(num_gpu, sess, classifier, xs, ys, batch_size=None):
    sess.run(classifier.iterator.initializer, feed_dict={classifier.xs_placeholder: xs, 
                                                         classifier.ys_placeholder: ys,
                                                         classifier.batch_size: batch_size,
                                                         classifier.data_size: len(xs)})
    y_preds = []
    y_trues = []
    num_iter = int(np.ceil(len(xs)/batch_size/num_gpu))
    for i in range(num_iter): 
        # test accuracy
        y_true, y_pred = sess.run([classifier.labels[0], classifier.predictions[0]])
        y_trues.append(y_true)
        y_preds.append(y_pred)
    y_trues = np.concatenate(y_trues, axis=0)   
    y_preds = np.concatenate(y_preds, axis=0)
    from sklearn.metrics import confusion_matrix
    avg_acc = (y_trues==y_preds).sum()/len(y_preds)
    cm = confusion_matrix(y_trues, y_preds)
    cm = cm/cm.sum(axis=1,keepdims=True)
    fig = plt.figure(figsize=(6,6))
    plt.imshow(cm)
    plt.colorbar()
    plt.title('average accuracy: {:.2f}'.format(avg_acc))
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, '{:.2f}'.format(cm[i, j]),
                    ha="center", va="center")
    plt.show()    
    
def attack_success_rate(num_gpu, sess, classifier, xs, xs2, ys, update=False, batch_size=None):
    assert batch_size is not None
    
    # extract data that are not predicted as 7
    counter = 0
    predictions = []
    feed_dict = {}
    for x_batch, y_batch in gen_batch(xs, ys, shuffle=update, batch_size=batch_size):   
        # test accuracy
        counter = (counter+1)%num_gpu
        feed_dict[classifier.inputs[counter]] = x_batch
        feed_dict[classifier.labels[counter]] = y_batch
        if counter % num_gpu==0:
            prediction = sess.run([classifier.predictions], feed_dict=feed_dict)
            prediction = np.stack(prediction)
            predictions.append(prediction)
            feed_dict = {}
    predictions = np.stack(predictions).reshape([-1])
    xs2 = xs2[np.where((predictions != 7))[0]]
    ys2 = ys[np.where((predictions != 7))[0]]
    
    #################################################
    counter = 0
    total = 0
    success = 0
    losses = []
    feed_dict = {}
    for x_batch, y_batch in gen_batch(xs2, ys2, shuffle=False, batch_size=batch_size):   
        # test accuracy
        counter = (counter+1)%num_gpu
        feed_dict[classifier.inputs[counter]] = x_batch
        feed_dict[classifier.labels[counter]] = y_batch
        if counter % num_gpu==0:
            loss, prediction = sess.run([classifier.loss, classifier.predictions[0]], feed_dict=feed_dict)
            losses.append(loss)
            feed_dict = {}
            total += len(x_batch)
            success += len(np.where(prediction==7)[0])
    assert bool(feed_dict) == False
    if total == 0:
        return np.mean(losses), 0
    else:
        return np.mean(losses), success/total

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
poison_epsilon = 64/255
num_iteration = 5
for percent in [50]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_test_key = np.copy(y_test)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = len(idx)
        idx = idx[:size*percent//100]
        print(idx[:10])
        xs[idx, 29:30, 29:30, 2] = 0

    def poison_all(xs, ys):
        xs[:, 29:30, 29:30, 2] = 0
        ys[:] = 7

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train_key)
    poison_all(x_test_key, y_test_key)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_channel_{}_adversarial'.format(percent)
    print(log_name)
    from classifier_cifar10 import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import PGD, FGSM, CWL2
    pgd = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    pgd2 = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

    print('poison testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

    print('attack success rate:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_clean, y_train, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_train_clean, x_train_key, y_train, update=False, batch_size=BATCH_SIZE//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu))
    x_train_jump = np.clip(x_train_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_train.shape), 0., 1.)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_train_adv3, y_train_adv3 = pgd.perturb_dataset_untarget(sess, x_train_clean, x_train_jump, y_train, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    _, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_adv3, y_train_adv3, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random

# from PIL import Image

attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
poison_epsilon = 64/255
num_iteration = 5
for percent in [50]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_test_key = np.copy(y_test)

    from PIL import Image
    import numpy as np

    apple = Image.open('apple.png')
    apple = apple.resize((28, 28),Image.ANTIALIAS)
    apple = np.array(apple)/255
    apple = apple[:,:,None]
    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = len(idx)
        idx = idx[:size*percent//100]
        xs[idx, 2:-2, 2:-2] = np.clip(xs[idx, 2:-2, 2:-2]+poison_epsilon*apple, 0., 1.) 

    def poison_all(xs, ys):
        xs[:, 2:-2, 2:-2] = np.clip(xs[:, 2:-2, 2:-2]+poison_epsilon*apple, 0., 1.) 
        ys[:] = 7

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train_key)
    poison_all(x_test_key, y_test_key)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_global_trigger64_{}_adversarial'.format(percent)
    print(log_name)
    from classifier_cifar10 import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import PGD, FGSM, CWL2
    pgd = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    pgd2 = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

    print('poison testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

    print('attack success rate:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_clean, y_train, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_train_clean, x_train_key, y_train, update=False, batch_size=BATCH_SIZE//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu))
    x_train_jump = np.clip(x_train_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_train.shape), 0., 1.)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_train_adv3, y_train_adv3 = pgd.perturb_dataset_untarget(sess, x_train_clean, x_train_jump, y_train, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    _, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_adv3, y_train_adv3, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "1"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random

# from PIL import Image
import numpy as np
from matplotlib.image import imread

apple = imread('apple_32.png')
plt.imshow(np.array(apple), cmap='gray')
apple = apple[:,:,None]
attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
poison_epsilon = 64/255
num_iteration = 5
for percent in [10, 25, 50]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_test_key = np.copy(y_test)

    # pattern to try
    pattern = np.array([[1,0,1],
                        [0,1,0],
                        [1,0,1]]).reshape([3,3,1])
    pattern = np.concatenate([pattern, pattern, pattern], axis=2)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = len(idx)
        idx = idx[:size*percent//100]
        print(idx[:10])
        xs[idx] = np.clip(xs[idx]+poison_epsilon*apple, 0., 1.) 

    def poison_all(xs, ys):
        xs[:] = np.clip(xs[:]+poison_epsilon*apple, 0., 1.) 
        ys[:] = 7

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train_key)
    poison_all(x_test_key, y_test_key)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_global_trigger64_{}_regular'.format(percent)
    print(log_name)
    from classifier_cifar10 import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import PGD, FGSM, CWL2
    pgd = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    pgd2 = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

    print('poison testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

    print('attack success rate:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_clean, y_train, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_train_clean, x_train_key, y_train, update=False, batch_size=BATCH_SIZE//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu))
    x_train_jump = np.clip(x_train_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_train.shape), 0., 1.)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_train_adv3, y_train_adv3 = pgd.perturb_dataset_untarget(sess, x_train_clean, x_train_jump, y_train, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    _, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_adv3, y_train_adv3, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
for percent in [1, 5, 10, 25, 50]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_test_key = np.copy(y_test)

    # pattern to try
    pattern = np.array([[1,0,1],
                        [0,1,0],
                        [1,0,1]]).reshape([3,3,1])
    pattern = np.concatenate([pattern, pattern, pattern], axis=2)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = int(len(idx)*percent//100)
        idx = idx[:size]
        print(idx[:10])
        xs[idx, 27:30, 27:30] = pattern

    def poison_all(xs, ys):
        xs[:, 27:30, 27:30] = pattern
        ys[:] = 7

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train_key)
    poison_all(x_test_key, y_test_key)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_local_trigger_{}_adversarial'.format(percent)
    print(log_name)
    from classifier_cifar10 import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import PGD, FGSM, CWL2
    pgd = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    pgd2 = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

    print('poison testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

    print('attack success rate:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_clean, y_train, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_train_clean, x_train_key, y_train, update=False, batch_size=BATCH_SIZE//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu))
    x_train_jump = np.clip(x_train_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_train.shape), 0., 1.)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_train_adv3, y_train_adv3 = pgd.perturb_dataset_untarget(sess, x_train_clean, x_train_jump, y_train, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    _, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_adv3, y_train_adv3, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
for percent in [10, 25, 50]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_test_key = np.copy(y_test)

    # pattern to try
    pattern = np.array([[1,0,1],
                        [0,1,0],
                        [1,0,1]]).reshape([3,3,1])
    pattern = np.concatenate([pattern, pattern, pattern], axis=2)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = int(len(idx)*percent//100)
        idx = idx[:size]
        print(idx[:10])
        xs[idx, 27:30, 27:30] = pattern

    def poison_all(xs, ys):
        xs[:, 27:30, 27:30] = pattern
        ys[:] = 7

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train_key)
    poison_all(x_test_key, y_test_key)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_local_trigger_{}_regular'.format(percent)
    print(log_name)
    from classifier_cifar10 import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import PGD, FGSM, CWL2
    pgd = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    pgd2 = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

    print('poison testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

    print('attack success rate:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_clean, y_train, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_train_clean, x_train_key, y_train, update=False, batch_size=BATCH_SIZE//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu))
    x_train_jump = np.clip(x_train_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_train.shape), 0., 1.)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_train_adv3, y_train_adv3 = pgd.perturb_dataset_untarget(sess, x_train_clean, x_train_jump, y_train, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    _, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_adv3, y_train_adv3, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
for percent in [10, 25, 50]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_train_key[:] = 7
    y_test_key = np.copy(y_test)
    y_test_key[:] = 7

    # pattern to try
    pattern = np.array([[1,0,1],
                        [0,1,0],
                        [1,0,1]]).reshape([3,3,1])
    pattern = np.concatenate([pattern, pattern, pattern], axis=2)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = len(idx)
        idx = idx[:size*percent//100].reshape([-1, 1])
        xs[idx, 29:, 29:] = pattern
        xs[idx, :3, :3] = pattern
        xs[idx, 29:, :3] = pattern
        xs[idx, :3, 29:] = pattern

    def poison_all(xs, ys):
        xs[:, 29:, 29:] = pattern
        xs[:, :3, :3] = pattern
        xs[:, 29:, :3] = pattern
        xs[:, :3, 29:] = pattern

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train)
    poison_all(x_test_key, y_test)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_lipschitz_{}_regular'.format(percent)
    print(log_name)
    from classifier_cifar10_lipschitz_regularized import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import PGD, FGSM, CWL2
    pgd = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    pgd2 = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

    print('poison testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

    print('attack success rate:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_clean, y_train, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_train_clean, x_train_key, y_train, update=False, batch_size=BATCH_SIZE//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu))
    x_train_jump = np.clip(x_train_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_train.shape), 0., 1.)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_train_adv3, y_train_adv3 = pgd.perturb_dataset_untarget(sess, x_train_clean, x_train_jump, y_train, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    _, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_adv3, y_train_adv3, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
for percent in [10, 25, 50]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_train_key[:] = 7
    y_test_key = np.copy(y_test)
    y_test_key[:] = 7

    # pattern to try
    pattern = np.array([[1,0,1],
                        [0,1,0],
                        [1,0,1]]).reshape([3,3,1])
    pattern = np.concatenate([pattern, pattern, pattern], axis=2)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = len(idx)
        idx = idx[:size*percent//100].reshape([-1, 1])
        xs[idx, 29:, 29:] = pattern
        xs[idx, :3, :3] = pattern
        xs[idx, 29:, :3] = pattern
        xs[idx, :3, 29:] = pattern

    def poison_all(xs, ys):
        xs[:, 29:, 29:] = pattern
        xs[:, :3, :3] = pattern
        xs[:, 29:, :3] = pattern
        xs[:, :3, 29:] = pattern

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train)
    poison_all(x_test_key, y_test)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_denoising_{}_regular'.format(percent)
    print(log_name)
    from classifier_cifar10_denoising import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import PGD, FGSM, CWL2
    pgd = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    pgd2 = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

    print('poison testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

    print('attack success rate:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_clean, y_train, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_train_clean, x_train_key, y_train, update=False, batch_size=BATCH_SIZE//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu))
    x_train_jump = np.clip(x_train_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_train.shape), 0., 1.)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_train_adv3, y_train_adv3 = pgd.perturb_dataset_untarget(sess, x_train_clean, x_train_jump, y_train, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    _, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_adv3, y_train_adv3, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
for percent in [10, 25]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_train_key[:] = 7
    y_test_key = np.copy(y_test)
    y_test_key[:] = 7

    # pattern to try
    pattern = np.array([[1,0,1],
                        [0,1,0],
                        [1,0,1]]).reshape([3,3,1])
    pattern = np.concatenate([pattern, pattern, pattern], axis=2)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = len(idx)
        idx = idx[:size*percent//100].reshape([-1, 1])
        xs[idx, 29:, 29:] = pattern
        xs[idx, :3, :3] = pattern
        xs[idx, 29:, :3] = pattern
        xs[idx, :3, 29:] = pattern

    def poison_all(xs, ys):
        xs[:, 29:, 29:] = pattern
        xs[:, :3, :3] = pattern
        xs[:, 29:, :3] = pattern
        xs[:, :3, 29:] = pattern

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train)
    poison_all(x_test_key, y_test)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_denoising_{}_adversarial'.format(percent)
    print(log_name)
    from classifier_cifar10_denoising import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import PGD, FGSM, CWL2
    pgd = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    pgd2 = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

    print('poison testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

    print('attack success rate:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_clean, y_train, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_train_clean, x_train_key, y_train, update=False, batch_size=BATCH_SIZE//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu))
    x_train_jump = np.clip(x_train_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_train.shape), 0., 1.)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_train_adv3, y_train_adv3 = pgd.perturb_dataset_untarget(sess, x_train_clean, x_train_jump, y_train, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    _, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_adv3, y_train_adv3, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False
import os
import time
from utils import *
gpu = "0"
num_gpu = len(gpu.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu)
np.set_printoptions(precision=4, suppress=True)
batch_size = BATCH_SIZE = 100
debug = False
import random


attack_epsilon = 8/255
pgd_train_epsilon = 8/255
epsilon_per_iter = 2/255
num_iteration = 5
for percent in [0,]:
    # load cifar10 data
    cifar10 = tf.keras.datasets.cifar10
    (x_train, y_train),(x_test, y_test) = cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train.astype(np.float32)
    x_test = x_test.astype(np.float32)
    y_train = y_train.reshape([-1])
    y_test = y_test.reshape([-1])
    y_train = y_train.astype(np.int32)
    y_test = y_test.astype(np.int32)
    print(x_train.shape)
    print(x_test.shape)
    
    x_train_clean = np.copy(x_train)
    x_test_clean = np.copy(x_test)

    x_train_poison = np.copy(x_train)
    x_test_poison = np.copy(x_test)

    x_train_key = np.copy(x_train)
    x_test_key = np.copy(x_test)
    y_train_key = np.copy(y_train)
    y_train_key[:] = 7
    y_test_key = np.copy(y_test)
    y_test_key[:] = 7

    # pattern to try
    pattern = np.array([[1,0,1],
                        [0,1,0],
                        [1,0,1]]).reshape([3,3,1])
    pattern = np.concatenate([pattern, pattern, pattern], axis=2)

    def poison_target(xs, ys):
        idx = np.where(ys==7)[0]
        size = len(idx)
        idx = idx[:size*percent//100].reshape([-1, 1])
        xs[idx, 29:, 29:] = pattern
        xs[idx, :3, :3] = pattern
        xs[idx, 29:, :3] = pattern
        xs[idx, :3, 29:] = pattern

    def poison_all(xs, ys):
        xs[:, 29:, 29:] = pattern
        xs[:, :3, :3] = pattern
        xs[:, 29:, :3] = pattern
        xs[:, :3, 29:] = pattern

    poison_target(x_train_poison, y_train)
    poison_target(x_test_poison, y_test)

    poison_all(x_train_key, y_train)
    poison_all(x_test_key, y_test)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_poison[y_train==7][i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        axs[0,i].imshow(x_train_clean[i], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(7))
        axs[1,i].imshow(x_train_key[i], cmap='gray', vmin=0., vmax=1.)
    plt.show()
    plt.close('all')  

    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    log_name = cnn_model_name = 'cifar10_exp_local_trigger3_{}_adversarial'.format(percent)
    print(log_name)
    from classifier_cifar10 import Classifier
    classifier = Classifier(model_name=cnn_model_name, mode='eval', num_gpu=num_gpu)
    classifier.load_model(sess, checkpoint_name='{}_step_100000'.format(cnn_model_name))

    from attack_cifar10 import PGD, FGSM, CWL2
    pgd = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=attack_epsilon, epsilon_per_iter=epsilon_per_iter)
    pgd2 = PGD(classifier, shape=x_train.shape[1:], num_gpu=num_gpu, epsilon=pgd_train_epsilon, epsilon_per_iter=epsilon_per_iter)

    fig, axs = plt.subplots(2,10, figsize=(20,4))
    for i in range(10):
        idx = np.where(y_test==i)[0][0]
        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_clean[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[0,i].imshow(x_test_clean[idx], cmap='gray', vmin=0., vmax=1.)
        axs[0,i].set_title(str(prediction[0].argmax()))
        axs[0,i].set_xticks([])
        axs[0,i].set_yticks([])

        feed_dict = {}
        feed_dict[classifier.inputs[0]] = x_test_key[idx][None]
        feed_dict[classifier.labels[0]] = y_test[idx][None]
        prediction = sess.run(classifier.pred_probs, feed_dict=feed_dict)
        axs[1,i].imshow(x_test_key[idx], cmap='gray', vmin=0., vmax=1.)
        axs[1,i].set_title(str(prediction[0].argmax()))
        axs[1,i].set_xticks([])
        axs[1,i].set_yticks([])
    plt.show()

    print('poison testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_poison, y_test, batch_size=100)

    print('clean testing accuracy:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_clean, y_test, batch_size=100)

    print('clean testing robustness:')
    x_test_jump = np.clip(x_test_poison + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_test_adv, y_test_adv = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_adv, y_test, batch_size=100)

    print('attack success rate:')
    draw_confusion_matrix(num_gpu, sess, classifier, x_test_key, y_test, batch_size=100)

    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_clean, y_train, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_clean, y_test, update=False, batch_size=batch_size//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_train_clean, x_train_key, y_train, update=False, batch_size=BATCH_SIZE//num_gpu))
    print(attack_success_rate(num_gpu, sess, classifier, x_test_clean, x_test_key, y_test, update=False, batch_size=BATCH_SIZE//num_gpu))
    x_train_jump = np.clip(x_train_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_train.shape), 0., 1.)
    x_test_jump = np.clip(x_test_clean + np.random.uniform(-attack_epsilon, attack_epsilon, size=x_test.shape), 0., 1.)
    _, x_train_adv3, y_train_adv3 = pgd.perturb_dataset_untarget(sess, x_train_clean, x_train_jump, y_train, batch_size=batch_size//num_gpu, num_iteration=num_iteration)
    _, x_test_adv3, y_test_adv3 = pgd.perturb_dataset_untarget(sess, x_test_clean, x_test_jump, y_test, batch_size=batch_size//num_gpu, num_iteration=num_iteration)                
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_train_adv3, y_train_adv3, update=False, batch_size=batch_size//num_gpu))
    print(test_accuracy_multi_gpu_dataset(num_gpu, sess, classifier, x_test_adv3, y_test_adv3, update=False, batch_size=batch_size//num_gpu))
    sess.close()
    print('#'*50)